In [3]:
import numpy as np
import nibabel
from loadnii import segment_labels
import pymesh # https://github.com/pmneila/PyMCubes
from cleanmesh import remove_empty_volume_around_mesh

from marching_tetrahedra import march_with_tetrahendras, export_triangles
import mcubes


### Load data

In [1]:
volume_file_path = "data/ctscan1/volume-94.nii.gz"
label_file_path = "data/ctscan1/labels-94.nii.gz"

isovalue = 0.8

"""
liver : 1 ,
bladder: 2, 
lungs : 3, 
kidneys: 4,
bones 5
"""


In [8]:

label_name = "bones_94"

label_id = 5

organs = {}
minlabel = label_id - .2
maxlabel = label_id + .2

new_image = segment_labels(volume_file_path, label_file_path, minlabel, maxlabel)

organs[label_name] = segmented_label_data = new_image.get_fdata()
organs[label_name] = remove_empty_volume_around_mesh(organs[label_name])
    
print(f"Loaded {label_name} of shape: {organs[label_name].shape} Seperation value was: {minlabel} - {maxlabel}")

Loaded bones_94 of shape: (455, 512, 917) Seperation value was: 4.8 - 5.2


### Marching Cubes

In [9]:
for organ_name, organ in organs.items():
    print(f"Marching trough {organ_name}...",end="")
    vertices, triangles = mcubes.marching_cubes(organ, isovalue)
    mesh = pymesh.form_mesh(vertices, triangles)
    pymesh.save_mesh(f"{organ_name}-marching_cubes.obj",mesh)
    print(f"done! Saved as: {organ_name}-marching_cubes.obj")
print("Finished march!")

Marching trough bones_94...done! Saved as: bones_94-marching_cubes.obj
Finished march!


### Marching tetrahendras

In [10]:
verbose = True

for organ_name, organ in organs.items():
    print(f"Marching trough {organ_name}...",end="")
    triangles = march_with_tetrahendras(organ, isovalue, verbose)
    export_triangles(triangles, f"{organ_name}-marching_tetrahendras") # Save as stl
    mesh = pymesh.load_mesh(f"{organ_name}-marching_tetrahendras.stl")
    pymesh.save_mesh(f"{organ_name}-marching_tetrahendras.obj",mesh)
    print(f"\nDone! saved as: {organ_name}-marching_tetrahendras.obj\n")
print("Finished march!")

Marching trough bones_94...Starting with march of volume (455, 512, 917)... 
I = Current iteration number, T = Current amount of found triangles
I:0 T:0, I:1 T:0, I:2 T:0, I:3 T:0, I:4 T:0, I:5 T:0, I:6 T:0, I:7 T:0, I:8 T:0, I:9 T:0, I:10 T:0, 
I:11 T:0, I:12 T:0, I:13 T:0, I:14 T:0, I:15 T:0, 
I:16 T:0, I:17 T:0, I:18 T:0, I:19 T:0, I:20 T:0, 
I:21 T:0, I:22 T:0, I:23 T:0, I:24 T:0, I:25 T:0, 
I:26 T:0, I:27 T:0, I:28 T:0, I:29 T:0, I:30 T:0, 
I:31 T:0, I:32 T:0, I:33 T:0, I:34 T:0, I:35 T:0, 
I:36 T:0, I:37 T:0, I:38 T:0, I:39 T:0, I:40 T:0, 
I:41 T:0, I:42 T:0, I:43 T:0, I:44 T:0, I:45 T:0, 
I:46 T:0, I:47 T:0, I:48 T:0, I:49 T:0, I:50 T:0, 
I:51 T:232, I:52 T:624, I:53 T:1086, I:54 T:1574, I:55 T:3522, 
I:56 T:6458, I:57 T:9668, I:58 T:12690, I:59 T:16430, I:60 T:22268, 
I:61 T:29968, I:62 T:37790, I:63 T:46286, I:64 T:54730, I:65 T:63586, 
I:66 T:74600, I:67 T:88400, I:68 T:103660, I:69 T:116872, I:70 T:131392, 
I:71 T:145928, I:72 T:159304, I:73 T:172984, I:74 T:187980, I:75 T:2

### Smoothing

In [ ]:
smooth_organs = {}
for organ_name, organ in organs.items():
    print(f"Smooting trough {organ_name}...",end="")
    smooth_organs[organ_name] = mcubes.smooth(organ)
    print(f"done!")
print("Finished Smooting!")

In [ ]:

for organ_name, organ in smooth_organs.items():
    print(f"Marching trough {organ_name}...",end="")
    vertices, triangles = mcubes.marching_cubes(organ, isovalue)
    mesh = pymesh.form_mesh(vertices, triangles)
    pymesh.save_mesh(f"{organ_name}-smooth-marching_cubes.obj",mesh)
    print(f"done! Saved as: {organ_name}-smooth-marching_cubes.obj")
print("Finished march!")

In [ ]:
verbose = True

for organ_name, organ in smooth_organs.items():
    print(f"Marching trough {organ_name}...",end="")
    triangles = march_with_tetrahendras(organ, isovalue, verbose)
    export_triangles(triangles, f"{organ_name}-smooth-marching_tetrahendras") # Save as stl
    mesh = pymesh.load_mesh(f"{organ_name}-smooth-marching_tetrahendras.stl")
    pymesh.save_mesh(f"{organ_name}-smooth-marching_tetrahendras.obj",mesh)
    print(f"Done! saved as: {organ_name}-smooth-marching_tetrahendras.obj")
print("Finished march!")

In [ ]:
from cleanmesh import cleanup_mesh

# mesh = cleanup_mesh(pymesh.load_mesh("bones-marching_tetrahendras.obj"))
pymesh.save_mesh("bones-marching_tetrahendras.obj", mesh)